In [1]:
### 通过query搜索主题并把内容同步到notion database中

## 生物
# Define your Notion API key and database ID
NOTION_DATABASE_ID = "3001b93dfe62442c90c96bf6fa888feb" #生物
# NOTION_DATABASE_ID = "8655c5459f8740a28e9f26f2f45a87dd" #agent
# NOTION_DATABASE_ID = "0dec242526284b4aad0de077ff4ccf30" #quant
# NOTION_DATABASE_ID = "85f5f98d2b3b48b3a49215c16525c28c" # 多模态大模型
# NOTION_DATABASE_ID = "fff5a4c02cd580d491a9ef076c6509e3" # neuroGPT

# Define the search query
query = "CombFold: predicting structures of large protein assemblies"

import os,sys
import requests

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root directory to the PYTHONPATH
if project_root not in sys.path:
    sys.path.append(project_root)

# Verify if the project root directory is in PYTHONPATH
print("Project root added to PYTHONPATH:", project_root in sys.path)

from open_science_agent.api.semantic_scholar_client import SemanticScholarClient
from open_science_agent.api.notion_client import NotionClient
from open_science_agent.data_processing.functions import submit_paper_to_notion

# Initialize the SemanticScholarClient
semantic_scholar_client = SemanticScholarClient(timeout=(30, 100))

# Initialize the NotionClient
notion_client = NotionClient()

# Perform the search by relevance
results = semantic_scholar_client.call_function_with_retries(semantic_scholar_client.search_papers_by_relevance,query)

# Iterate over the results and store them in the Notion database
for paper in results.get("data", []):
    submit_paper_to_notion(paper, notion_client, NOTION_DATABASE_ID) 
    recommended_papers = semantic_scholar_client.call_function_with_retries(semantic_scholar_client.get_recommended_papers,results['data'][0]['paperId'])
    for recommended_paper in recommended_papers.get('recommendedPapers',[]):
        submit_paper_to_notion(recommended_paper, notion_client, NOTION_DATABASE_ID) 

print("All papers have been added to the Notion database.")


Project root added to PYTHONPATH: True
HTTP error occurred: 429 Client Error:  for url: https://api.semanticscholar.org/graph/v1/paper/search?query=CombFold%3A+predicting+structures+of+large+protein+assemblies&offset=0&limit=20&fields=corpusId%2Curl%2Ctitle%2Cvenue%2CpublicationVenue%2Cyear%2Cauthors%2CexternalIds%2Cabstract%2CreferenceCount%2CcitationCount%2CinfluentialCitationCount%2CisOpenAccess%2CopenAccessPdf%2CfieldsOfStudy%2Cs2FieldsOfStudy%2CpublicationTypes%2CpublicationDate%2Cjournal%2CcitationStyles%2Ctldr&minCitationCount=0&year=2020-
Response content: b'{"message": "Too Many Requests. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form", "code": "429"}'
Attempt 1 failed: 429 Client Error:  for url: https://api.semanticscholar.org/graph/v1/paper/search?query=CombFold%3A+predicting+structures+of+large+protein+assemblies&offset=0&limit=20&fields=corpusId%2Curl%2Ctitle%2Cvenue%2CpublicationVenue%2Cyear%2

In [4]:
import os
import sys
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output

# 将open_science_agent库添加到Python路径中
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from open_science_agent.api.semantic_scholar_client import SemanticScholarClient
from open_science_agent.api.notion_client import NotionClient
from open_science_agent.data_processing.functions import map_paper_to_notion_properties

# 初始化SemanticScholarClient和NotionClient
semantic_scholar_client = SemanticScholarClient(timeout=(30, 100))
notion_client = NotionClient()

# 创建Notion数据库ID的下拉菜单
notion_db_options = {
    "生物": "3001b93dfe62442c90c96bf6fa888feb",
    "Agent": "8655c5459f8740a28e9f26f2f45a87dd",
    "Quant": "0dec242526284b4aad0de077ff4ccf30",
    "多模态大模型": "85f5f98d2b3b48b3a49215c16525c28c",
    "医学大模型": "fff5a4c02cd580d491a9ef076c6509e3"
}

notion_db_dropdown = widgets.Dropdown(
    options=notion_db_options,
    description='Notion DB:',
    disabled=False,
)

# 创建搜索query的文本输入框
query_input = widgets.Text(
    value='neuroscience neuroimage llm',
    placeholder='Type your search query here',
    description='Query:',
    disabled=False
)

# 创建执行按钮
execute_button = widgets.Button(
    description='Execute',
    disabled=False,
    button_style='', 
    tooltip='Click to execute the search and sync with Notion',
    icon='check'
)

# 定义按钮点击事件的处理函数
def on_execute_button_clicked(b):
    clear_output(wait=True)
    
    # 选择的Notion数据库ID
    NOTION_DATABASE_ID = notion_db_dropdown.value
    
    # 输入的搜索query
    query = query_input.value
    
    print(f"Selected Notion Database ID: {NOTION_DATABASE_ID}")
    print(f"Search Query: {query}")
    
    # 执行搜索并同步到Notion数据库
    results = semantic_scholar_client.get_search_results_with_retries(query)
    
    for paper in results.get("data", []):
        properties = map_paper_to_notion_properties(paper)
        
        filter_criteria = {"property": "PaperId", "rich_text": {"equals": paper.get('paperId')}}
        query_results = notion_client.query_database(NOTION_DATABASE_ID, filter_criteria=filter_criteria)
        
        if len(query_results['results']) == 0:
            feed_back = notion_client.create_page(database_id=NOTION_DATABASE_ID, properties=properties)
            print('Added:', paper.get("title", {}))
        else:
            print('Already added:', paper.get("title", {}))

    print("All papers have been added to the Notion database.")

# 绑定按钮点击事件处理函数
execute_button.on_click(on_execute_button_clicked)

# 显示交互式小部件
display(notion_db_dropdown, query_input, execute_button)


Dropdown(description='Notion DB:', options={'生物': '3001b93dfe62442c90c96bf6fa888feb', 'Agent': '8655c5459f8740…

Text(value='neuroscience neuroimage llm', description='Query:', placeholder='Type your search query here')

Button(description='Execute', icon='check', style=ButtonStyle(), tooltip='Click to execute the search and sync…

In [12]:
properties.get('PaperId')

{'rich_text': [{'text': {'content': 'a20411effeaac9aa457e528090dc274cb46c3412'}}]}

In [15]:

# 定义过滤器和排序条件
filter_criteria = {"property": "PaperId","rich_text": {"equals": paper.get('paperId')}}
# 查询数据库
results = notion_client.query_database(NOTION_DATABASE_ID, filter_criteria=filter_criteria)

# 处理结果
print(results)

{'object': 'list', 'results': [{'object': 'page', 'id': 'e12b2d75-c70d-4f0f-bde1-00b24df946cd', 'created_time': '2024-08-13T09:36:00.000Z', 'last_edited_time': '2024-08-13T09:36:00.000Z', 'created_by': {'object': 'user', 'id': '6d558a8e-25e6-4afa-af8b-ae2dc29f5c2a'}, 'last_edited_by': {'object': 'user', 'id': '6d558a8e-25e6-4afa-af8b-ae2dc29f5c2a'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '3001b93d-fe62-442c-90c9-6bf6fa888feb'}, 'archived': False, 'in_trash': False, 'properties': {'FieldsOfStudy': {'id': '%3C_On', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': '["Medicine"]', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': '["Medicine"]', 'href': None}]}, 'PublicationVenue': {'id': '%3DR_m', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': '{"id": "099483df-e8f2-4bee-805d-8a69f07b6cbf", "name":

In [9]:
len(results['results'])

0

In [4]:
results

{'total': 0, 'offset': 0}

### 交互式文献搜索
下面的代码实现了一个交互式的程序，用户可以通过输入查询关键字，动态获取并显示相关的学术论文结果。

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# 导入SemanticScholarClient（假设它在open_science_agent包中）
from open_science_agent.api.semantic_scholar_client import SemanticScholarClient

semantic_scholar_client = SemanticScholarClient()

# 定义查询输入框和按钮
query_input = widgets.Text(
    value='machine learning in atmospheric science',
    placeholder='Enter search query',
    description='Query:',
    disabled=False
)

search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Search for papers',
    icon='search'
)

output = widgets.Output()

# 定义搜索函数
def search_papers(b):
    query = query_input.value
    results = semantic_scholar_client.search_papers_by_relevance(query)
    
    # 提取结果数据
    data = results['data']
    papers = []
    for paper in data:
        papers.append({
            'Title': paper['title'],
            'Authors': ', '.join([author['name'] for author in paper['authors']]),
            'Year': paper['year'],
            'Venue': paper['venue'],
            'Abstract': paper.get('abstract', 'No abstract available'),
            'Citations': paper['citationCount'],
            'URL': paper['url']
        })
    
    # 转换为DataFrame
    df = pd.DataFrame(papers)
    
    # 在输出框中显示结果
    with output:
        clear_output(wait=True)
        display(df)

# 绑定搜索按钮点击事件
search_button.on_click(search_papers)

# 显示查询输入框和按钮
display(query_input, search_button, output)

Text(value='machine learning in atmospheric science', description='Query:', placeholder='Enter search query')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Search for papers')

Output()

ConnectTimeout: HTTPSConnectionPool(host='api.semanticscholar.org', port=443): Max retries exceeded with url: /graph/v1/paper/search?query=llava&offset=0&limit=20&fields=corpusId%2Curl%2Ctitle%2Cvenue%2CpublicationVenue%2Cyear%2Cauthors%2CexternalIds%2Cabstract%2CreferenceCount%2CcitationCount%2CinfluentialCitationCount%2CisOpenAccess%2CopenAccessPdf%2CfieldsOfStudy%2Cs2FieldsOfStudy%2CpublicationTypes%2CpublicationDate%2Cjournal%2CcitationStyles%2Ctldr&publicationTypes=Review%2CJournalArticle&minCitationCount=5 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001817ED359A0>, 'Connection to api.semanticscholar.org timed out. (connect timeout=None)'))

: 

In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# 导入SemanticScholarClient（假设它在open_science_agent包中）
from open_science_agent.api.semantic_scholar_client import SemanticScholarClient

# 初始化SemanticScholarClient
semantic_scholar_client = SemanticScholarClient()

# 定义查询输入框和按钮
query_input = widgets.Text(
    value='machine learning in atmospheric science',
    placeholder='请输入查询关键字',
    description='查询:',
    disabled=False
)

search_button = widgets.Button(
    description='搜索',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='搜索学术论文',
    icon='search'
)

output = widgets.Output()
details_output = widgets.Output()

# 定义显示详细信息的函数
def show_paper_details(selected_paper):
    with details_output:
        clear_output(wait=True)
        display(pd.DataFrame([selected_paper]))

# 定义搜索函数
def search_papers(b):
    query = query_input.value
    results = semantic_scholar_client.search_papers_by_relevance(query)
    
    # 提取结果数据
    data = results['data']
    papers = []
    for paper in data:
        papers.append({
            '标题': paper['title'],
            '作者': ', '.join([author['name'] for author in paper['authors']]),
            '年份': paper['year'],
            '期刊': paper['venue'],
            '摘要': paper.get('abstract', '没有摘要'),
            '引用数': paper['citationCount'],
            '链接': paper['url'],
            '详细信息': paper  # 添加整个论文数据，以便后续显示详细信息
        })
    
    # 转换为DataFrame
    df = pd.DataFrame(papers)
    
    # 创建交互表格
    table = widgets.Output()
    with table:
        clear_output(wait=True)
        display(df)
    
    # 创建选择器
    dropdown = widgets.Dropdown(
        options=[(paper['标题'], i) for i, paper in enumerate(papers)],
        description='选择论文:',
        disabled=False
    )
    
    # 定义选择事件处理函数
    def on_select(change):
        selected_paper = papers[change['new']]['详细信息']
        show_paper_details(selected_paper)
    
    dropdown.observe(on_select, names='value')
    
    # 显示查询结果和选择器
    with output:
        clear_output(wait=True)
        display(table, dropdown)

# 绑定搜索按钮点击事件
search_button.on_click(search_papers)

# 显示查询输入框和按钮
display(query_input, search_button, output, details_output)

Text(value='machine learning in atmospheric science', description='查询:', placeholder='请输入查询关键字')

Button(description='搜索', icon='search', style=ButtonStyle(), tooltip='搜索学术论文')

Output()

Output()

In [4]:
import requests  
from bs4 import BeautifulSoup  
from urllib.parse import urljoin  

def extract_images_and_tables(url):  
    # 发送GET请求获取网页内容  
    response = requests.get(url)  
    response.raise_for_status()  # 检查请求是否成功  

    # 使用BeautifulSoup解析网页内容  
    soup = BeautifulSoup(response.content, 'html.parser')  

    # 提取所有图片的URL  
    images = soup.find_all('img')  
    image_urls = [urljoin(url, img['src']) for img in images if img.get('src')]  

    # 提取所有表格  
    tables = soup.find_all('table')  
    table_htmls = [str(table) for table in tables]  

    return image_urls, table_htmls  

# 示例用法  
url = 'https://www.semanticscholar.org/paper/a9eab711118774d1b897f70cbb0f420da144fe85'  # 替换为你要处理的URL  
images, tables = extract_images_and_tables(url)  

print("Images found:")  
for img in images:  
    print(img)  

print("\nTables found:")  
for table in tables:  
    print(table)  

Images found:

Tables found:


In [5]:
import requests
from bs4 import BeautifulSoup
import os

def fetch_figures_and_tables(url):
    # Fetch the webpage content
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful

    # Parse the webpage content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Directory to save images
    if not os.path.exists('images'):
        os.makedirs('images')

    # Extract figures and tables along with captions
    figures_tables = []
    
    # Find all image objects that are associated with figures or tables
    for img_tag in soup.find_all('img'):
        # Check if it is a figure or table based on the caption or alt text
        caption = img_tag.get('alt', 'No caption')
        src = img_tag['src']

        # Handle relative URLs by joining with the base URL
        if not src.startswith('http'):
            src = os.path.join(os.path.dirname(url), src)

        # Download the image
        img_data = requests.get(src).content
        img_name = os.path.join('images', os.path.basename(src))
        with open(img_name, 'wb') as handler:
            handler.write(img_data)

        figures_tables.append({'caption': caption, 'image': img_name})

    return figures_tables

# Example usage:
url = 'https://www.semanticscholar.org/paper/A-Comprehensive-Review-of-AI-in-Healthcare%3A-Neural-Sathe-Deodhe/a9eab711118774d1b897f70cbb0f420da144fe85'
figures_and_tables = fetch_figures_and_tables(url)

# Print out the results
for item in figures_and_tables:
    print(f"Caption: {item['caption']}")
    print(f"Image File: {item['image']}")